# GDVSpectra CVA module tests

## Globals

In [49]:
# set globals paths
FOLDER_MODULES = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules'  
FOLDER_SHARED = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared'
TEST_MODULE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\tests\code'
GRP_LYR_FILE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx'    

## Setup

### Imports

In [52]:
# imports
import os
import datetime
import random
import numpy as np
import xarray as xr
from IPython.utils import io

# import testing functions
sys.path.append(TEST_MODULE)
import test_funcs

# import full arcpy toolbox
arcpy.ImportToolbox(r"C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\toolbox\tenement-tools-toolbox.pyt")

<module 'toolbox'>

### Reload libraries

In [51]:
# if scripts change, reload
from importlib import reload
reload(test_funcs)

<module 'test_funcs' from 'C:\\Users\\Lewis\\Documents\\GitHub\\tenement-tools\\tests\\code\\test_funcs.py'>

### Set data files and locations

In [53]:
# setup general io
input_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs'
output_folder = r'E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\outputs'

# temp nc file and temp mask nc file for use when breaking ncs, mask ncs
temp_nc = os.path.join(input_folder, 'temp_nc.nc')               # temp nc file for use when breaking ncs
temp_mask_nc = os.path.join(input_folder, 'temp_mask_nc.nc')     # temp nc file for use when breaking ncs

# setup landsat cubes paths
ls_cubes = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_1_ls_90_20_cva_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_2_ls_90_20_cva_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_3_ls_90_20_cva_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_4_ls_90_20_cva_odc.nc",
]

# setup sentinel2 cubes paths
s2_cubes = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_1_s2_16_20_cva_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_2_s2_16_20_cva_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_3_s2_16_20_cva_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_4_s2_16_20_cva_odc.nc",
]

# setup landsat cubes mask paths
ls_masks = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_1_ls_90_20_cva_mask_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_2_ls_90_20_cva_mask_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_3_ls_90_20_cva_mask_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_4_ls_90_20_cva_mask_odc.nc",
]

# setup sentinel2 cubes mask paths
s2_masks = [
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_1_s2_16_20_cva_mask_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_2_s2_16_20_cva_mask_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_3_s2_16_20_cva_mask_odc.nc",
    r"E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_4_s2_16_20_cva_mask_odc.nc",
]

### Set specific raw netcdf and shapefiles

In [54]:
# set top level netcdf and shapefile
nc_file = ls_cubes[1]
nc_mask_file = ls_masks[1]

### Set up function to iterate corruptor and tests (without mask support)

In [55]:
def run_corruptors_through_tests(in_nc, nc_corrs, tests, verbose):
    """
    this func takes a path to nc file or raw sat imagery, a list of nc
    corruptor funcs and params, a list of test funcs and params. Each 
    nc corruptor func is iterated through, and for each corrupted nc, 
    each test in tests is applied to corrupted nc. verbose sets how
    much information is printed.
    """
    
    for nc_corr in nc_corrs:
        corr_name = nc_corr[0].__name__                   # name of current corruptor func
        corr_func, corr_params = nc_corr[0], nc_corr[1]   # pointer to corruptor func and dict of params

        # notify
        print('Corrupting NetCDF via: {}.\n'.format(corr_name) + '- ' * 30)

        # create temp nc and corrupt it with current corruptor
        if not verbose:
            with io.capture_output() as cap:
                test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)
        else:
            test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)

        # run current corruptor function
        try:
            corr_func(**corr_params)
        except Exception as e:    
            print(e)
            print('Corruptor did not have enough data to work with. Skipping.\n')

        # iter each test func and apply to current corrupt nc
        for test in tests:
            test_nc_name = corr_name + '_' + test[0]    # name of current test nc
            test_func, test_params = test[1], test[2]   # pointer to test func and dict of params
            test_msg = test[3]
            
            # notify of test message
            print(test_msg)

            # create output nc file path and name and update params for in/out paths
            out_nc_file = os.path.join(output_folder, test_nc_name)
            
            # remove output nc if exists
            if os.path.exists(out_nc_file):
                os.remove(out_nc_file)
            
            # update params
            test_params.update({'in_raw_nc_path': temp_nc, 'out_nc_path': out_nc_file})

            # perform current test
            try:
                # notify
                print('Performing test: {}.'.format(test_nc_name))

                # perform test, provide prints if requested
                if not verbose:
                    with io.capture_output() as cap:
                        test_func(**test_params)
                else:
                    test_func(**test_params)
                    print('\n')

            except Exception as e:    
                print(e)

        # notify
        print('All tests applied to corruptor NetCDF.\n\n')

### Set up function to iterate corruptor and tests (with mask support)

In [56]:
def run_corruptors_through_tests_with_mask(in_nc, in_mask_nc, nc_corrs, mask_corrs, tests, verbose):
    """
    this func takes a path to nc file or raw sat imagery, a list of nc
    corruptor funcs and params, a list of test funcs and params. Each 
    nc corruptor func is iterated through, and for each corrupted nc, 
    each test in tests is applied to corrupted nc. verbose sets how
    much information is printed. considers mask corruptors.
    """
    
    for nc_corr in nc_corrs:
        nc_corr_name = nc_corr[0].__name__                      # name of current corruptor func
        nc_corr_func, nc_corr_params = nc_corr[0], nc_corr[1]   # pointer to corruptor func and dict of params

        # notify
        print('Corrupting NetCDF via: {}.\n'.format(nc_corr_name) + '- ' * 30)

        # create temp nc and corrupt it with current corruptor
        if not verbose:
            with io.capture_output() as cap:
                test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)
        else:
            test_funcs.create_temp_nc(in_nc=in_nc, out_nc=temp_nc)

        # run current corruptor function
        try:
            nc_corr_func(**nc_corr_params)
        except Exception as e:    
            print(e)
            print('NetCDF Corruptor did not have enough data to work with. Skipping.\n')
            
        # now, we iterate each mask nc corruptor for this current corrupt nc!
        for mask_corr in mask_corrs:
            mask_corr_name = mask_corr[0].__name__                          # name of current corruptor func
            mask_corr_func, mask_corr_params = mask_corr[0], mask_corr[1]   # pointer to corruptor func and dict of params
            
            # notify
            print('Corrupting mask NetCDF via: {}.\n'.format(mask_corr_name) + '- ' * 30)
            
            # create temp nc and corrupt it with current corruptor
            if not verbose:
                with io.capture_output() as cap:
                    test_funcs.create_temp_nc(in_nc=in_mask_nc, out_nc=temp_mask_nc)
            else:
                test_funcs.create_temp_nc(in_nc=in_mask_nc, out_nc=temp_mask_nc)
            
            # run current corruptor function
            try:
                mask_corr_func(**mask_corr_params)
            except Exception as e:    
                print(e)
                print('Mask NetCDF Corruptor did not have enough data to work with. Skipping.\n')
            
            # iter each test func and apply to current corrupt nc with consideration to shape
            for test in tests:
                test_nc_name = nc_corr_name + '_' + test[0]    # name of current test nc
                test_func, test_params = test[1], test[2]      # pointer to test func and dict of params
                test_msg = test[3]

                # notify of test message
                print(test_msg)
                
                # create output nc file path and name and update params for in/out paths
                out_nc_file = os.path.join(output_folder, test_nc_name)

                # remove output nc if exists
                if os.path.exists(out_nc_file):
                    os.remove(out_nc_file)
                    
                # update params
                test_params.update({'in_raw_nc_path': temp_nc, 'out_nc_path': out_nc_file, 'in_mask_nc_path': temp_mask_nc})

                # perform current test
                try:
                    # notify
                    print('Performing test: {}.'.format(test_nc_name))

                    # perform test, provide prints if requested
                    if not verbose:
                        with io.capture_output() as cap:
                            test_func(**test_params)
                    else:
                        test_func(**test_params)
                        print('\n')

                except Exception as e:    
                    print(e)

            # notify
            print('All tests applied to corruptor NetCDF.\n\n')

### Set up netcdf corruptor functions

In [57]:
# these are numerous netcdf corruptors. feed a raw nc in, break it, output as temp nc
# comment out any that are irrelevant
# each of these uncommented will be fed through the tests below
def build_nc_corruptors(temp_nc):
    """
    each one of these is a unique netcdf corruptor functions and 
    associated parameters. 
    """
    
    # set up list
    cs = []

    # func: raw default dataset, no changes
    cs.append([test_funcs.nc_default, {'in_nc': temp_nc}])

#     # func: remove x, y, time, spatial_ref coords
#     cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'x'}])
#     cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'y'}])
#     cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'time'}])
#     cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'spatial_ref'}])

#     #func: remove red and oa_fmask band vars
#     cs.append([test_funcs.remove_var, {'in_nc': temp_nc, 'var': 'like'}])

#     #func: limit number of years in various combos
#     cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 1990, 'e_year': 1990}])
#     cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2010, 'e_year': 2010}])
#     cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2012, 'e_year': 2012}])
#     cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 1991, 'e_year': 1992}])
#     cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2005, 'e_year': 2006}])
#     cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2019, 'e_year': 2020}])
#     cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 1993, 'e_year': 1995}])
#     cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2010, 'e_year': 2012}])
#     cs.append([test_funcs.limit_years, {'in_nc': temp_nc, 's_year': 2011, 'e_year': 2013}])

#     #func: set all vars to nan
#     cs.append([test_funcs.set_nc_vars_all_nan, {'in_nc': temp_nc}])

#     #func: set all vars to zero
#     cs.append([test_funcs.set_nc_vars_all_zero, {'in_nc': temp_nc}])

#     #func: set all vars for 10 rand times to all nan
#     cs.append([test_funcs.set_nc_vars_random_all_nan, {'in_nc': temp_nc, 'num': 10}])

#     #func: strip all attrs from nc    
#     cs.append([test_funcs.strip_nc_attributes, {'in_nc': temp_nc}])

#     #func: set vars in first and last time index to all nan
#     cs.append([test_funcs.set_end_times_to_all_nan, {'in_nc': temp_nc}])

#     #func: reduce whole nc to one random time slice
#     cs.append([test_funcs.reduce_to_one_scene, {'in_nc': temp_nc}])

#     #func: set months all nan, all years, for specific months
#     cs.append([test_funcs.set_all_specific_season_nan, {'in_nc': temp_nc, 'months': [1]}])

#     #func: set months all nan, specific years, for specific months
#     cs.append([test_funcs.set_specific_years_season_nan, {'in_nc': temp_nc, 'years': [1990], 'months': [1]}])
#     cs.append([test_funcs.set_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2005], 'months': [1]}])
#     cs.append([test_funcs.set_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2006, 2007], 'months': [1]}])

#     #func: drop wet months, all years, for specific months
#     cs.append([test_funcs.remove_all_specific_season_nan, {'in_nc': temp_nc, 'months': [1]}])

#     #func: drop wet months, specific years, for specific months (note: seperate tests)
#     cs.append([test_funcs.remove_specific_years_season_nan, {'in_nc': temp_nc, 'years': [1990], 'months': [1]}])
#     cs.append([test_funcs.remove_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2009], 'months': [1]}])
#     cs.append([test_funcs.remove_specific_years_season_nan, {'in_nc': temp_nc, 'years': [2007, 2008], 'months': [1]}])

#     #func: remove crs attribute
#     cs.append([test_funcs.remove_crs_attr, {'in_nc': temp_nc}])

#     #func: invalidate crs attribute
#     cs.append([test_funcs.invalidate_crs_attr, {'in_nc': temp_nc, 'crs_text': 'EPSG:4326'}])
#     cs.append([test_funcs.invalidate_crs_attr, {'in_nc': temp_nc, 'crs_text': ''}])

#     #func: remove nodatavals attribute
#     cs.append([test_funcs.remove_nodatavals_attr, {'in_nc': temp_nc}])
    
    return cs

nc_corruptors = build_nc_corruptors(temp_nc=temp_nc)

### Setup mask corruptor functions

In [58]:
# these are numerous mask nc corruptors. feed a raw shp in, break it, output as temp nc
# comment out any that are irrelevant
# each of these uncommented will be fed through some tests below
def build_mask_corruptors(temp_mask_nc):
    """
    each one of these is a unique netcdf corruptor functions and 
    associated parameters. 
    """
    
    # set up list
    cs = []

    # func: raw default dataset, no changes
    cs.append([test_funcs.nc_default, {'in_nc': temp_mask_nc}])
    
#     # func: remove x, y, time, spatial_ref coords
#     cs.append([test_funcs.remove_coord, {'in_nc': temp_mask_nc, 'coord': 'x'}])
#     cs.append([test_funcs.remove_coord, {'in_nc': temp_mask_nc, 'coord': 'y'}])
#     cs.append([test_funcs.remove_coord, {'in_nc': temp_nc, 'coord': 'spatial_ref'}])

#     #func: remove red and oa_fmask band vars
#     cs.append([test_funcs.remove_var, {'in_nc': temp_nc, 'var': 'like'}])

#     #func: set all vars to nan
#     cs.append([test_funcs.set_nc_vars_all_nan, {'in_nc': temp_nc}])

#     #func: set all vars to zero
#     cs.append([test_funcs.set_nc_vars_all_zero, {'in_nc': temp_nc}])

#     #func: set all vars to ones
#     cs.append([test_funcs.set_nc_vars_all_ones, {'in_nc': temp_nc}])

#     #func: strip all attrs from nc    
#     cs.append([test_funcs.strip_nc_attributes, {'in_nc': temp_nc}])

#     #func: remove crs attribute
#     cs.append([test_funcs.remove_crs_attr, {'in_nc': temp_nc}])

#     #func: invalidate crs attribute
#     cs.append([test_funcs.invalidate_crs_attr, {'in_nc': temp_nc, 'crs_text': 'EPSG:4326'}])
#     cs.append([test_funcs.invalidate_crs_attr, {'in_nc': temp_nc, 'crs_text': ''}])

#     #func: remove nodatavals attribute
#     cs.append([test_funcs.remove_nodatavals_attr, {'in_nc': temp_nc}])

#     # func: shift dataset by coords
#     cs.append([test_funcs.shift_dataset_via_coords, {'in_nc': temp_mask_nc, 'x': 300, 'y': 500}])
#     cs.append([test_funcs.shift_dataset_via_coords, {'in_nc': temp_mask_nc, 'x': 5000, 'y': 5000}])

#     # func: reduce reslution
#     cs.append([test_funcs.reduce_resolution, {'in_nc': temp_mask_nc, 'factor': 2}])
    
#     # func: multiply values by a lot
#     cs.append([test_funcs.multiply_values, {'in_nc': temp_mask_nc, 'factor': 50}])    
    
    return cs


## Run tests

### Test One: Mask NetCDF

In [62]:
def build_test_one_funcs(in_nc, temp_nc):
    """sets up test one functions"""
    
    # set default params for tool
    inputs = {
        'in_raw_nc_path': '',                      # input like nc (i.e. temp nc)
        'in_mask_nc_path': '',                     # input mask nc (i.e. temp mask nc)
        'out_nc_path': '',                         # output nc (i.e. t1a nc)
        'in_base_start_year': None,                # base start year
        'in_base_end_year': None,                  # base end year
        'in_comp_start_year': None,                # comparison start year
        'in_comp_end_year': None,                  # comparison end year
        'in_analysis_months': [9, 10, 11],         # analysis months
        'in_tmf': 2.0,                             # magnitude threshold
        'in_stand_qupper': 0.99,                   # upper ivt value
        'in_stand_qlower': 0.05,                   # lower ivt value
        'in_fmask_flags': 'Valid;Snow;Water',      # fmask flag values
        'in_max_cloud': 10,                        # max cloud percentage
        'in_interpolate': True,                    # interpolate missing pixels
        'in_add_result_to_map': True               # add result to map
    }
    
    # set up list
    ts = []

    # func: default like and mask ncs provided
    msg = 'Running Test One: default like nc with default mask input.'
    params = inputs.copy()
    ts.append(['t_1_def.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    return ts

### Test One: Run!

In [64]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
mask_corrs = build_mask_corruptors(temp_mask_nc)
tests = build_test_one_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests_with_mask(nc_file, nc_mask_file, nc_corrs, mask_corrs, tests, verbose=True)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Duplicating cube: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_2_ls_90_20_cva_odc.nc
No changes, setting up for default dataset.
Corrupting mask NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Duplicating cube: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_2_ls_90_20_cva_mask_odc.nc
No changes, setting up for default dataset.
Running Test One: default like nc with default mask input.
Performing test: nc_default_t_1_def.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask

### Test Two: From Date - Base Start and End Years

In [75]:
def build_test_two_funcs(in_nc, temp_nc):
    """sets up test two functions"""
    
    # set default params for tool
    inputs = {
        'in_raw_nc_path': '',                      # input like nc (i.e. temp nc)
        'in_mask_nc_path': '',                     # input mask nc (i.e. temp mask nc)
        'out_nc_path': '',                         # output nc (i.e. t1a nc)
        'in_base_start_year': None,                # base start year
        'in_base_end_year': None,                  # base end year
        'in_comp_start_year': 2015,                # comparison start year
        'in_comp_end_year': 2020,                  # comparison end year
        'in_analysis_months': [9, 10, 11],         # analysis months
        'in_tmf': 2.0,                             # magnitude threshold
        'in_stand_qupper': 0.99,                   # upper ivt value
        'in_stand_qlower': 0.05,                   # lower ivt value
        'in_fmask_flags': 'Valid;Snow;Water',      # fmask flag values
        'in_max_cloud': 10,                        # max cloud percentage
        'in_interpolate': True,                    # interpolate missing pixels
        'in_add_result_to_map': True,              # add result to map
    }
    
    # set up list
    ts = []

    # func: default to and from dates without mask
    msg = 'Running Test Two: default base start and end yers without mask input.'
    params = inputs.copy()
    params.update({'in_base_start_year': None, 'in_base_end_year': None})
    ts.append(['t_2_def.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: default to and from dates without mask
    msg = 'Running Test Two: base start and end yers 1990, 2000 without mask input.'
    params = inputs.copy()
    params.update({'in_base_start_year': 1990, 'in_base_end_year': 2000})
    ts.append(['t_2_a.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: default to and from dates without mask
    msg = 'Running Test Two: base start and end yers 2000, 2010 without mask input.'
    params = inputs.copy()
    params.update({'in_base_start_year': 2000, 'in_base_end_year': 2010})
    ts.append(['t_2_b.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: default to and from dates without mask
    msg = 'Running Test Two: base start and end yers 2000, 2015 without mask input.'
    params = inputs.copy()
    params.update({'in_base_start_year': 2000, 'in_base_end_year': 2015})
    ts.append(['t_2_c.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: default to and from dates without mask
    msg = 'Running Test Two: base start higher than end (2000, 1990) without mask input.'
    params = inputs.copy()
    params.update({'in_base_start_year': 2000, 'in_base_end_year': 1990})
    ts.append(['t_2_d.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: default to and from dates without mask
    msg = 'Running Test Two: base start == end (2000, 2000) without mask input.'
    params = inputs.copy()
    params.update({'in_base_start_year': 2000, 'in_base_end_year': 2000})
    ts.append(['t_2_e.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    
 
    # func: default to and from dates without mask
    msg = 'Running Test Two: base start fine and end None (2000, None) without mask input.'
    params = inputs.copy()
    params.update({'in_base_start_year': 2000, 'in_base_end_year': None})
    ts.append(['t_2_f.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])   
    
    # func: default to and from dates without mask but different compariuson years
    msg = 'Running Test Two: default base start and end yers without mask input but 90-19 comp years.'
    params = inputs.copy()
    params.update({'in_base_start_year': None, 'in_base_end_year': None, 
                   'in_comp_start_year': 1990, 'in_comp_end_year': 2019})
    ts.append(['t_2_g.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: default to and from dates without mask but different compariuson years
    msg = 'Running Test Two: default base start and end yers without mask input but 95-00 comp years.'
    params = inputs.copy()
    params.update({'in_base_start_year': None, 'in_base_end_year': None, 
                   'in_comp_start_year': 1995, 'in_comp_end_year': 2000})
    ts.append(['t_2_h.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: default to and from dates without mask but different compariuson years
    msg = 'Running Test Two: default base start and end yers without mask input but None, None comp years.'
    params = inputs.copy()
    params.update({'in_base_start_year': None, 'in_base_end_year': None, 
                   'in_comp_start_year': None, 'in_comp_end_year': None})
    ts.append(['t_2_i.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: default to and from dates without mask but different compariuson years
    msg = 'Running Test Two: default base start and end yers without mask input but 1990-2020 comp years.'
    params = inputs.copy()
    params.update({'in_base_start_year': None, 'in_base_end_year': None, 
                   'in_comp_start_year': 1990, 'in_comp_end_year': 2020})
    ts.append(['t_2_i.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # try above with default mask nc now 
    inputs.update({'in_mask_nc_path': nc_mask_file, 
                   'in_comp_start_year': 2015, 'in_comp_end_year': 2020})
    
    # func: default to and from dates with mask
    msg = 'Running Test Two: default base start and end yers with mask input.'
    params = inputs.copy()
    params.update({'in_base_start_year': None, 'in_base_end_year': None})
    ts.append(['t_2_g.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: to and from dates with mask
    msg = 'Running Test Two: base start and end yers 2011, 2019 with mask input.'
    params = inputs.copy()
    params.update({'in_base_start_year': 2011, 'in_base_end_year': 2019})
    ts.append(['t_2_h.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: to and from dates with mask
    msg = 'Running Test Two: base start and end yers 2019, 2010 with mask input.'
    params = inputs.copy()
    params.update({'in_base_start_year': 2019, 'in_base_end_year': 2010})
    ts.append(['t_2_i.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])   
        
    return ts

### Test Two: Run!

In [76]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_two_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_corrs=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Two: default base start and end yers without mask input.
Performing test: nc_default_t_2_def.nc.
Running Test Two: base start and end yers 1990, 2000 without mask input.
Performing test: nc_default_t_2_a.nc.
Running Test Two: base start and end yers 2000, 2010 without mask input.
Performing test: nc_default_t_2_b.nc.
Running Test Two: base start and end yers 2000, 2015 without mask input.
Performing test: nc_default_t_2_c.nc.
Running Test Two: base start higher than end (2000, 1990) without mask input.
Performing test: nc_default_t_2_d.nc.
Running Test Two: base start == end (2000, 2000) without mask input.
Performing test: nc_default_t_2_e.nc.
Running Test Two: base start fine and end None (2000, None) without mask input.
Performing test: nc_default_t_2_f.nc.
Running Test Two: default base start and end yers without mask input but 90-

### Test Three: From Date - Comparison Start and End Years

In [77]:
def build_test_three_funcs(in_nc, temp_nc):
    """sets up test three functions"""
    
    # set default params for tool
    inputs = {
        'in_raw_nc_path': '',                      # input like nc (i.e. temp nc)
        'in_mask_nc_path': '',                     # input mask nc (i.e. temp mask nc)
        'out_nc_path': '',                         # output nc (i.e. t1a nc)
        'in_base_start_year': 1990,                # base start year
        'in_base_end_year': 2010,                  # base end year
        'in_comp_start_year': None,                # comparison start year
        'in_comp_end_year': None,                  # comparison end year
        'in_analysis_months': [9, 10, 11],         # analysis months
        'in_tmf': 2.0,                             # magnitude threshold
        'in_stand_qupper': 0.99,                   # upper ivt value
        'in_stand_qlower': 0.05,                   # lower ivt value
        'in_fmask_flags': 'Valid;Snow;Water',      # fmask flag values
        'in_max_cloud': 10,                        # max cloud percentage
        'in_interpolate': True,                    # interpolate missing pixels
        'in_add_result_to_map': True,              # add result to map
    }
    
    # set up list
    ts = []

    # func: default to and from dates without mask
    msg = 'Running Test Three: default comparison start and end yers without mask input.'
    params = inputs.copy()
    params.update({'in_comp_start_year': None, 'in_comp_end_year': None})
    ts.append(['t_3_def.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: to and from dates without mask)
    msg = 'Running Test Three: comparison start and end yers (2010, 2020) without mask input.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 2010, 'in_comp_end_year': 2020})
    ts.append(['t_3_a.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: to and from dates without mask)
    msg = 'Running Test Three: comparison start and end yers (2015, 2000) without mask input.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 2015, 'in_comp_end_year': 2000})
    ts.append(['t_3_b.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: to and from dates without mask)
    msg = 'Running Test Three: comparison start and end yers (1990, 2000) without mask input.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 1990, 'in_comp_end_year': 2000})
    ts.append(['t_3_c.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: to and from dates without mask)
    msg = 'Running Test Three: comparison start and end yers (1990, 2020) without mask input.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 1990, 'in_comp_end_year': 2020})
    ts.append(['t_3_d.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    

    # func: to and from dates without mask)
    msg = 'Running Test Three: comparison start and end yers (2015, 2015) without mask input.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 2015, 'in_comp_end_year': 2015})
    ts.append(['t_3_e.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    
    
    # func: to and from dates without mask)
    msg = 'Running Test Three: comparison start and end yers (2003, None) without mask input.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 2003, 'in_comp_end_year': None})
    ts.append(['t_3_f.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    
    
    # func: to and from dates without mask)
    msg = 'Running Test Three: comparison start and end yers (2010, 2020) without mask input but base yers 1995, 1994.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 2010, 'in_comp_end_year': 2020,
                   'in_base_start_year': 1995, 'in_base_end_year': 1994})
    ts.append(['t_3_g.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    

    # func: to and from dates without mask)
    msg = 'Running Test Three: comparison start and end yers (2010, 2015) without mask input but base yers 2010, 2015.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 2010, 'in_comp_end_year': 2015,
                   'in_base_start_year': 2010, 'in_base_end_year': 2015})
    ts.append(['t_3_h.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    
    
    # func: to and from dates without mask)
    msg = 'Running Test Three: comparison start and end yers (2000, 2005) without mask input but base yers 1990, 1995.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 2000, 'in_comp_end_year': 2005,
                   'in_base_start_year': 1990, 'in_base_end_year': 1995})
    ts.append(['t_3_i.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    
    
    # func: to and from dates without mask)
    msg = 'Running Test Three: comparison start and end yers (2019, 2019) without mask input but base yers 2019, 2019.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 2019, 'in_comp_end_year': 2019,
                   'in_base_start_year': 2019, 'in_base_end_year': 2019})
    ts.append(['t_3_j.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg]) 
    
    # func: to and from dates without mask)
    msg = 'Running Test Three: comparison start and end yers (2019, 2019) without mask input but base yers 2019, 2019.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 2017, 'in_comp_end_year': 2018,
                   'in_base_start_year': 2019, 'in_base_end_year': 2020})
    ts.append(['t_3_k.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg]) 
    
    # try above with default mask nc now 
    inputs.update({'in_mask_nc_path': nc_mask_file, 'in_base_start_year': 2010, 'in_base_end_year': 2015})
    
    # func: default to and from dates with mask
    msg = 'Running Test Three: default comp start and end yers with mask input.'
    params = inputs.copy()
    params.update({'in_comp_start_year': None, 'in_comp_end_year': None})
    ts.append(['t_2_l.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: to and from dates with mask
    msg = 'Running Test Three: comp start and end yers (90-20) with mask input.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 1990, 'in_comp_end_year': 2020})
    ts.append(['t_2_m.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: to and from dates with mask
    msg = 'Running Test Three: comp start and end yers (19-19) with mask input.'
    params = inputs.copy()
    params.update({'in_comp_start_year': 2019, 'in_comp_end_year': 2019})
    ts.append(['t_2_n.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

        
    return ts

### Test Three: Run!

In [78]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_three_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_corrs=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Three: default comparison start and end yers without mask input.
Performing test: nc_default_t_3_def.nc.
Running Test Three: comparison start and end yers (2010, 2020) without mask input.
Performing test: nc_default_t_3_a.nc.
Running Test Three: comparison start and end yers (2015, 2000) without mask input.
Performing test: nc_default_t_3_b.nc.
Running Test Three: comparison start and end yers (1990, 2000) without mask input.
Performing test: nc_default_t_3_c.nc.
Running Test Three: comparison start and end yers (1990, 2020) without mask input.
Performing test: nc_default_t_3_d.nc.
Running Test Three: comparison start and end yers (2015, 2015) without mask input.
Performing test: nc_default_t_3_e.nc.
Running Test Three: comparison start and end yers (2003, None) without mask input.
Performing test: nc_default_t_3_f.nc.
Running Test Thr

### Test Four:  Analysis Months

In [82]:
def build_test_four_funcs(in_nc, temp_nc):
    """sets up test four functions"""
    
    # set default params for tool
    inputs = {
        'in_raw_nc_path': '',                      # input like nc (i.e. temp nc)
        'in_mask_nc_path': '',                     # input mask nc (i.e. temp mask nc)
        'out_nc_path': '',                         # output nc (i.e. t1a nc)
        'in_base_start_year': 1990,                # base start year
        'in_base_end_year': 2005,                  # base end year
        'in_comp_start_year': 2005,                # comparison start year
        'in_comp_end_year': 2020,                  # comparison end year
        'in_analysis_months': None,                # analysis months
        'in_tmf': 2.0,                             # magnitude threshold
        'in_stand_qupper': 0.99,                   # upper ivt value
        'in_stand_qlower': 0.05,                   # lower ivt value
        'in_fmask_flags': 'Valid;Snow;Water',      # fmask flag values
        'in_max_cloud': 10,                        # max cloud percentage
        'in_interpolate': True,                    # interpolate missing pixels
        'in_add_result_to_map': True,              # add result to map
    }
    
    # set up list
    ts = []

    # func: default analysis months without mask
    msg = 'Running Test Four: default analysis months without mask.'
    params = inputs.copy()
    params.update({'in_analysis_months': '9;10;11'})
    ts.append(['t_4_def.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: analysis months without mask
    msg = 'Running Test Four: analysis months 1, 2, 3 without mask.'
    params = inputs.copy()
    params.update({'in_analysis_months': '1;2;3'})
    ts.append(['t_4_a.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: analysis months without mask
    msg = 'Running Test Four: analysis 4 without mask.'
    params = inputs.copy()
    params.update({'in_analysis_months': '4'})
    ts.append(['t_4_b.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: analysis months without mask
    msg = 'Running Test Four: analysis 0 without mask.'
    params = inputs.copy()
    params.update({'in_analysis_months': '0'})
    ts.append(['t_4_c.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: analysis months without mask
    msg = 'Running Test Four: analysis 9, 9 without mask.'
    params = inputs.copy()
    params.update({'in_analysis_months': '9;9'})
    ts.append(['t_4_d.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: analysis months without mask
    msg = 'Running Test Four: analysis 1-12 without mask.'
    params = inputs.copy()
    params.update({'in_analysis_months': '1;2;3;4;5;6;7;8;9;10;11;12'})
    ts.append(['t_4_e.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    
    
    # func: analysis months without mask
    msg = 'Running Test Four: analysis 12, 1, 2 without mask.'
    params = inputs.copy()
    params.update({'in_analysis_months': '12;1;2'})
    ts.append(['t_4_f.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])        
    
    # func: analysis months without mask
    msg = 'Running Test Four: analysis "" without mask.'
    params = inputs.copy()
    params.update({'in_analysis_months': ''})
    ts.append(['t_4_g.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg]) 
     
    # try above with default mask nc now 
    inputs.update({'in_mask_nc_path': nc_mask_file})
    
    # func: default analysis months with mask
    msg = 'Running Test Four: default analysis months with mask.'
    params = inputs.copy()
    params.update({'in_analysis_months': '9;10;11'})
    ts.append(['t_4_h.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: default analysis months with mask
    msg = 'Running Test Four: analysis months 1-12 with mask.'
    params = inputs.copy()
    params.update({'in_analysis_months': '1;2;3;4;5;6;7;8;9;10;11;12'})
    ts.append(['t_4_i.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    
    
    # func: default analysis months with mask
    msg = 'Running Test Four: analysis months 3, 4 with mask.'
    params = inputs.copy()
    params.update({'in_analysis_months': '3;4'})
    ts.append(['t_4_j.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    
        
    return ts

### Test Four: Run!

In [83]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_four_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_corrs=nc_corrs, tests=tests, verbose=True)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Duplicating cube: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_2_ls_90_20_cva_odc.nc
No changes, setting up for default dataset.
Running Test Four: default analysis months without mask.
Performing test: nc_default_t_4_def.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.
Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.
Subsetting down to specified months.
Reducing dataset into months: [9, 10, 11].
Subset to requested months successfully.
Calculating indices:

Isolated CVA angles successfully.
Isolating CVA angles from 180-270 degrees.
Isolated CVA angles successfully.
Isolating CVA angles from 270-360 degrees.
Isolated CVA angles successfully.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_2_ls_90_20_cva_mask_odc.nc
Netcdf converted to xarray dataset successfully.
Exporting xarray as netcdf file.
Exported xarray as netcdf successfully.


Running Test Four: analysis months 3, 4 with mask.
Performing test: nc_default_t_4_j.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.


### Test Five: TMF

In [85]:
def build_test_five_funcs(in_nc, temp_nc):
    """sets up test five functions"""
    
    # set default params for tool
    inputs = {
        'in_raw_nc_path': '',                      # input like nc (i.e. temp nc)
        'in_mask_nc_path': '',                     # input mask nc (i.e. temp mask nc)
        'out_nc_path': '',                         # output nc (i.e. t1a nc)
        'in_base_start_year': 1990,                # base start year
        'in_base_end_year': 2005,                  # base end year
        'in_comp_start_year': 2005,                # comparison start year
        'in_comp_end_year': 2020,                  # comparison end year
        'in_analysis_months': [9, 10, 11],         # analysis months
        'in_tmf': None,                            # magnitude threshold
        'in_stand_qupper': 0.99,                   # upper ivt value
        'in_stand_qlower': 0.05,                   # lower ivt value
        'in_fmask_flags': 'Valid;Snow;Water',      # fmask flag values
        'in_max_cloud': 10,                        # max cloud percentage
        'in_interpolate': True,                    # interpolate missing pixels
        'in_add_result_to_map': True,              # add result to map
    }
    
    # set up list
    ts = []

    # func: default tmf without mask
    msg = 'Running Test Five: default tmf without mask.'
    params = inputs.copy()
    params.update({'in_tmf': 2.0})
    ts.append(['t_5_def.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: tmf without mask
    msg = 'Running Test Five: tmf 1.0 without mask.'
    params = inputs.copy()
    params.update({'in_tmf': 1.0})
    ts.append(['t_5_a.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: tmf without mask
    msg = 'Running Test Five: tmf 0.0 without mask.'
    params = inputs.copy()
    params.update({'in_tmf': 0.0})
    ts.append(['t_5_b.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    
    
    # func: tmf without mask
    msg = 'Running Test Five: tmf -2 without mask.'
    params = inputs.copy()
    params.update({'in_tmf': -2.0})
    ts.append(['t_5_c.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])      
    
    # func: tmf without mask
    msg = 'Running Test Five: tmf 5 without mask.'
    params = inputs.copy()
    params.update({'in_tmf': 5.0})
    ts.append(['t_5_d.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])      
    
    # func: tmf without mask
    msg = 'Running Test Five: tmf 100 without mask.'
    params = inputs.copy()
    params.update({'in_tmf': 1.0})
    ts.append(['t_5_e.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])          
    
    # func: tmf without mask
    msg = 'Running Test Five: tmf 100 without mask.'
    params = inputs.copy()
    params.update({'in_tmf': None})
    ts.append(['t_5_f.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])       
     
    # try above with default mask nc now 
    inputs.update({'in_mask_nc_path': nc_mask_file})
    
    # func: default tmf with mask
    msg = 'Running Test Five: default tmf with mask.'
    params = inputs.copy()
    params.update({'in_tmf': 2.0})
    ts.append(['t_5_g.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: default tmf with mask
    msg = 'Running Test Five: tmf 0.0 with mask.'
    params = inputs.copy()
    params.update({'in_tmf': 0.0})
    ts.append(['t_5_h.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: default tmf with mask
    msg = 'Running Test Five: tmf 10.0 with mask.'
    params = inputs.copy()
    params.update({'in_tmf': 10.0})
    ts.append(['t_5_i.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
        
    return ts

### Test Five: Run!

In [86]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_five_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_corrs=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Five: default tmf without mask.
Performing test: nc_default_t_5_def.nc.
Running Test Five: tmf 1.0 without mask.
Performing test: nc_default_t_5_a.nc.
Running Test Five: tmf 0.0 without mask.
Performing test: nc_default_t_5_b.nc.
Running Test Five: tmf -2 without mask.
Performing test: nc_default_t_5_c.nc.
Failed to execute. Parameters are not valid.
ERROR 000854: The value is out of the range from 0 to 100.
Failed to execute (GDVSpectra_CVA).

Running Test Five: tmf 5 without mask.
Performing test: nc_default_t_5_d.nc.
Running Test Five: tmf 100 without mask.
Performing test: nc_default_t_5_e.nc.
Running Test Five: tmf 100 without mask.
Performing test: nc_default_t_5_f.nc.
Running Test Five: default tmf with mask.
Performing test: nc_default_t_5_g.nc.
Running Test Five: tmf 0.0 with mask.
Performing test: nc_default_t_5_h.nc.
Running

### Test Six: IVT Upper and Lower

In [88]:
def build_test_six_funcs(in_nc, temp_nc):
    """sets up test six functions"""
    
    # set default params for tool
    inputs = {
        'in_raw_nc_path': '',                      # input like nc (i.e. temp nc)
        'in_mask_nc_path': '',                     # input mask nc (i.e. temp mask nc)
        'out_nc_path': '',                         # output nc (i.e. t1a nc)
        'in_base_start_year': 1990,                # base start year
        'in_base_end_year': 2005,                  # base end year
        'in_comp_start_year': 2005,                # comparison start year
        'in_comp_end_year': 2020,                  # comparison end year
        'in_analysis_months': [9, 10, 11],         # analysis months
        'in_tmf': 2.0,                             # magnitude threshold
        'in_stand_qupper': None,                   # upper ivt value
        'in_stand_qlower': None,                   # lower ivt value
        'in_fmask_flags': 'Valid;Snow;Water',      # fmask flag values
        'in_max_cloud': 10,                        # max cloud percentage
        'in_interpolate': True,                    # interpolate missing pixels
        'in_add_result_to_map': True,              # add result to map
    }
    
    # set up list
    ts = []

    # func: default tmf without mask
    msg = 'Running Test Six: default ivt upper and lower without mask.'
    params = inputs.copy()
    params.update({'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05})
    ts.append(['t_6_def.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: tmf without mask
    msg = 'Running Test Six: ivt upper and lower "", "" without mask.'
    params = inputs.copy()
    params.update({'in_stand_qupper': '', 'in_stand_qlower': ''})
    ts.append(['t_6_a.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])  

    # func: tmf without mask
    msg = 'Running Test Six: ivt upper and lower 0.0, 0.99 without mask.'
    params = inputs.copy()
    params.update({'in_stand_qupper': 0.0, 'in_stand_qlower': 0.99})
    ts.append(['t_6_b.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])  
    
    # func: tmf without mask
    msg = 'Running Test Six: ivt upper and lower 0.05, 0.0 without mask.'
    params = inputs.copy()
    params.update({'in_stand_qupper': 0.05, 'in_stand_qlower': 0.0})
    ts.append(['t_6_c.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])      
    
    # func: tmf without mask
    msg = 'Running Test Six: ivt upper and lower 0.25, 0.80 without mask.'
    params = inputs.copy()
    params.update({'in_stand_qupper': 0.25, 'in_stand_qlower': 0.80})
    ts.append(['t_6_d.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])  
    
    # func: tmf without mask
    msg = 'Running Test Six: ivt upper and lower 0.4, 0.6 without mask.'
    params = inputs.copy()
    params.update({'in_stand_qupper': 0.4, 'in_stand_qlower': 0.6})
    ts.append(['t_6_e.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])   
    
    # func: tmf without mask
    msg = 'Running Test Six: ivt upper and lower 0.5, 0.5 without mask.'
    params = inputs.copy()
    params.update({'in_stand_qupper': 0.5, 'in_stand_qlower': 0.5})
    ts.append(['t_6_f.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])  

    # func: tmf without mask
    msg = 'Running Test Six: ivt upper and lower "0.2", "0.8" without mask.'
    params = inputs.copy()
    params.update({'in_stand_qupper': '0.2', 'in_stand_qlower': '0.8'})
    ts.append(['t_6_g.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])   

    # try above with default mask nc now 
    inputs.update({'in_mask_nc_path': nc_mask_file})
    
    # func: default tmf without mask
    msg = 'Running Test Six: default ivt upper and lower with mask.'
    params = inputs.copy()
    params.update({'in_stand_qupper': 0.99, 'in_stand_qlower': 0.05})
    ts.append(['t_6_h.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: tmf with mask
    msg = 'Running Test Six: ivt upper and lower 0.3, 0.7 with mask.'
    params = inputs.copy()
    params.update({'in_stand_qupper': 0.3, 'in_stand_qlower': 0.7})
    ts.append(['t_6_i.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    
    
    # func: tmf with mask
    msg = 'Running Test Six: ivt upper and lower 0.5, 0.5 with mask.'
    params = inputs.copy()
    params.update({'in_stand_qupper': 0.5, 'in_stand_qlower': 0.5})
    ts.append(['t_6_j.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])   
        
    return ts

### Test Six: Run!

In [89]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_six_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_corrs=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Duplicating cube: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\yandi_2_ls_90_20_cva_odc.nc
No changes, setting up for default dataset.
Running Test Six: default ivt upper and lower without mask.
Performing test: nc_default_t_6_def.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.
Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.
Subsetting down to specified months.
Reducing dataset into months: [9, 10, 11].
Subset to requested months successfully.
Calculating indic

Traceback (most recent call last):
  File "<string>", line 4139, in execute
  File "C:\Users\Lewis\Documents\GitHub\tenement-tools\modules\gdvspectra.py", line 1267, in standardise_to_targets
    raise ValueError('Upper quantile value must be larger than lower quantile.')
ValueError: Upper quantile value must be larger than lower quantile.


ERROR 000582: Error occurred during execution.

Running Test Six: ivt upper and lower 0.05, 0.0 without mask.
Performing test: nc_default_t_6_c.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.
Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.
Subsetting down to specified months.
Reducing dataset into months: [9, 10, 11].
Subset to requested months successfully.
Calculating indices: ['tcg', 'tcb'].
Calculating index: tcg
Calculating index: tcb
Calculated indices successfully.
Resampling dataset down to annual medians.
Resampled down to annual medians successfully.
Interpolating empty va

Traceback (most recent call last):
  File "<string>", line 4139, in execute
  File "C:\Users\Lewis\Documents\GitHub\tenement-tools\modules\gdvspectra.py", line 1267, in standardise_to_targets
    raise ValueError('Upper quantile value must be larger than lower quantile.')
ValueError: Upper quantile value must be larger than lower quantile.


ERROR 000582: Error occurred during execution.

Running Test Six: ivt upper and lower 0.4, 0.6 without mask.
Performing test: nc_default_t_6_e.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.
Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.
Subsetting down to specified months.
Reducing dataset into months: [9, 10, 11].
Subset to requested months successfully.
Calculating indices: ['tcg', 'tcb'].
Calculating index: tcg
Calculating index: tcb
Calculated indices successfully.
Resampling dataset down to annual medians.
Resampled down to annual medians successfully.
Interpolating empty val

...

ERROR 000582: Error occurred during execution.

Running Test Six: default ivt upper and lower with mask.
Performing test: nc_default_t_6_h.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.
Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.
Subsetting down to specified months.
Reducing dataset into months: [9, 10, 11].
Subset to requested months successfully.
Calculating indices: ['tcg', 'tcb'].
Calculating index: tcg
Calculating index: tcb
Calculated indices successfully.
Resampling dataset down to annual medians.
Resampled down to annual medians successfully.
Interpolating empty values 

Traceback (most recent call last):
  File "<string>", line 4139, in execute
  File "C:\Users\Lewis\Documents\GitHub\tenement-tools\modules\gdvspectra.py", line 1267, in standardise_to_targets
    raise ValueError('Upper quantile value must be larger than lower quantile.')
ValueError: Upper quantile value must be larger than lower quantile.


ERROR 000582: Error occurred during execution.

Running Test Six: ivt upper and lower 0.5, 0.5 with mask.
Performing test: nc_default_t_6_j.nc.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: E:\Curtin\GDVII - General\Work Package 2\test_data\gdvspectra_cva\inputs\temp_nc.nc
Netcdf converted to xarray dataset successfully.
Removing dates where too many invalid pixels.
Mask band is currently dask. Computing, please wait.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.
Conforming DEA ARD satellite band names.
Satellite band names conformed successfully.
Subsetting down to specified months.
Reducing dataset into months: [9, 10, 11].
Subset to requested months successfully.
Calculating indices: ['tcg', 'tcb'].
Calculating index: tcg
Calculating index: tcb
Calculated indices successfully.
Resampling dataset down to annual medians.
Resampled down to annual medians successfully.
Interpolating empty values

Traceback (most recent call last):
  File "<string>", line 4139, in execute
  File "C:\Users\Lewis\Documents\GitHub\tenement-tools\modules\gdvspectra.py", line 1267, in standardise_to_targets
    raise ValueError('Upper quantile value must be larger than lower quantile.')
ValueError: Upper quantile value must be larger than lower quantile.


ERROR 000582: Error occurred during execution.

All tests applied to corruptor NetCDF.




### Test Seven: Flags

In [90]:
def build_test_seven_funcs(in_nc, temp_nc):
    """sets up test seven functions"""
    
    # set default params for tool
    inputs = {
        'in_raw_nc_path': '',                      # input like nc (i.e. temp nc)
        'in_mask_nc_path': '',                     # input mask nc (i.e. temp mask nc)
        'out_nc_path': '',                         # output nc (i.e. t1a nc)
        'in_base_start_year': 1990,                # base start year
        'in_base_end_year': 2005,                  # base end year
        'in_comp_start_year': 2005,                # comparison start year
        'in_comp_end_year': 2020,                  # comparison end year
        'in_analysis_months': [9, 10, 11],         # analysis months
        'in_tmf': 2.0,                             # magnitude threshold
        'in_stand_qupper': 0.99,                   # upper ivt value
        'in_stand_qlower': 0.05,                   # lower ivt value
        'in_fmask_flags': None,                    # fmask flag values
        'in_max_cloud': 10,                        # max cloud percentage
        'in_interpolate': True,                    # interpolate missing pixels
        'in_add_result_to_map': True,              # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default pixel flags
    msg = 'Running Test Seven: pixel flags Valid;Snow;Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Valid;Snow;Water'})
    ts.append(['t_7_def.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: pixel flags is ''
    msg = 'Running Test Seven: pixel flags is "".'
    params = inputs.copy()
    params.update({'in_fmask_flags': ''})
    ts.append(['t_7_a.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: pixel flags is NoData;Valid;Cloud;Shadow;Snow;Water
    msg = 'Running Test Seven: pixel flags is NoData;Valid;Cloud;Shadow;Snow;Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'NoData;Valid;Cloud;Shadow;Snow;Water'})
    ts.append(['t_7_b.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    

    # func: pixel flags is Valid
    msg = 'Running Test Seven: pixel flags is Valid.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Valid'})
    ts.append(['t_7_c.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg]) 

    # func: pixel flags is Cloud;Shadow
    msg = 'Running Test Seven: pixel flags is Cloud;Shadow.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Cloud;Shadow'})
    ts.append(['t_7_d.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg]) 

    # func: pixel flags is Water
    msg = 'Running Test Seven: pixel flags is Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Water'})
    ts.append(['t_7_e.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg]) 

    # func: pixel flags is Water;Water
    msg = 'Running Test Seven: pixel flags is Water;Water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'Water;Water'})
    ts.append(['t_7_f.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg]) 

    # func: pixel flags is water
    msg = 'Running Test Seven: pixel flags is water.'
    params = inputs.copy()
    params.update({'in_fmask_flags': 'water'})
    ts.append(['t_7_g.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg]) 
    
    return ts

### Test Seven: Run!

In [91]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_seven_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_corrs=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Seven: pixel flags Valid;Snow;Water.
Performing test: nc_default_t_7_def.nc.
Running Test Seven: pixel flags is "".
Performing test: nc_default_t_7_a.nc.
Failed to execute. Parameters are not valid.
ERROR 000735: Include pixel flags: Value is required
Failed to execute (GDVSpectra_CVA).

Running Test Seven: pixel flags is NoData;Valid;Cloud;Shadow;Snow;Water.
Performing test: nc_default_t_7_b.nc.
Running Test Seven: pixel flags is Valid.
Performing test: nc_default_t_7_c.nc.
Running Test Seven: pixel flags is Cloud;Shadow.
Performing test: nc_default_t_7_d.nc.
Running Test Seven: pixel flags is Water.
Performing test: nc_default_t_7_e.nc.
NetCDF is completely null. Please add more years.
Failed to execute (GDVSpectra_CVA).

Running Test Seven: pixel flags is Water;Water.
Performing test: nc_default_t_7_f.nc.
Duplicate pixel flags detec

### Test Eight: Cloud Cover

In [93]:
def build_test_eight_funcs(in_nc, temp_nc):
    """sets up test eight functions"""
    
    # set default params for tool
    inputs = {
        'in_raw_nc_path': '',                      # input like nc (i.e. temp nc)
        'in_mask_nc_path': '',                     # input mask nc (i.e. temp mask nc)
        'out_nc_path': '',                         # output nc (i.e. t1a nc)
        'in_base_start_year': 1990,                # base start year
        'in_base_end_year': 2005,                  # base end year
        'in_comp_start_year': 2005,                # comparison start year
        'in_comp_end_year': 2020,                  # comparison end year
        'in_analysis_months': [9, 10, 11],         # analysis months
        'in_tmf': 2.0,                             # magnitude threshold
        'in_stand_qupper': 0.99,                   # upper ivt value
        'in_stand_qlower': 0.05,                   # lower ivt value
        'in_fmask_flags': 'Valid;Snow;Water',      # fmask flag values
        'in_max_cloud': None,                        # max cloud percentage
        'in_interpolate': True,                    # interpolate missing pixels
        'in_add_result_to_map': True,              # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default max cloud cover
    msg = 'Running Test Eight: default max cloud (10).'
    params = inputs.copy()
    params.update({'in_max_cloud': 10})
    ts.append(['t_8_def.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])

    # func: max cloud cover is 0
    msg = 'Running Test Eight: max cloud cover is 0.'
    params = inputs.copy()
    params.update({'in_max_cloud': 0})
    ts.append(['t_8_a.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: max cloud cover is 50
    msg = 'Running Test Eight: max cloud cover is 50.'
    params = inputs.copy()
    params.update({'in_max_cloud': 50})
    ts.append(['t_8_b.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    

    # func: max cloud cover is "10"
    msg = 'Running Test Eight: max cloud cover is "10".'
    params = inputs.copy()
    params.update({'in_max_cloud': "10"})
    ts.append(['t_8_c.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])  
    
    # func: max cloud cover is "10"
    msg = 'Running Test Eight: max cloud cover is "10".'
    params = inputs.copy()
    params.update({'in_max_cloud': "10"})
    ts.append(['t_8_d.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])      

    # func: max cloud cover is 150
    msg = 'Running Test Eight: max cloud cover is 150.'
    params = inputs.copy()
    params.update({'in_max_cloud': 150})
    ts.append(['t_8_e.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])   
    
    # func: max cloud cover is ''
    msg = 'Running Test Eight: max cloud cover is "".'
    params = inputs.copy()
    params.update({'in_max_cloud': ''})
    ts.append(['t_8_f.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])   

    # func: max cloud cover is None
    msg = 'Running Test Eight: max cloud cover is None.'
    params = inputs.copy()
    params.update({'in_max_cloud': None})
    ts.append(['t_8_g.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])  
    
    return ts

### Test Eight: Run!

In [94]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_eight_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_corrs=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Eight: default max cloud (10).
Performing test: nc_default_t_8_def.nc.
Running Test Eight: max cloud cover is 0.
Performing test: nc_default_t_8_a.nc.
Running Test Eight: max cloud cover is 50.
Performing test: nc_default_t_8_b.nc.
Running Test Eight: max cloud cover is "10".
Performing test: nc_default_t_8_c.nc.
Running Test Eight: max cloud cover is "10".
Performing test: nc_default_t_8_d.nc.
Running Test Eight: max cloud cover is 150.
Performing test: nc_default_t_8_e.nc.
Failed to execute. Parameters are not valid.
ERROR 000854: The value is out of the range from 0 to 100.
Failed to execute (GDVSpectra_CVA).

Running Test Eight: max cloud cover is "".
Performing test: nc_default_t_8_f.nc.
Running Test Eight: max cloud cover is None.
Performing test: nc_default_t_8_g.nc.
All tests applied to corruptor NetCDF.




### Test Nine: Interpolate

In [ ]:
def build_test_nine_funcs(in_nc, temp_nc):
    """sets up test nine functions"""
    
    # set default params for tool
    inputs = {
        'in_raw_nc_path': '',                      # input like nc (i.e. temp nc)
        'in_mask_nc_path': '',                     # input mask nc (i.e. temp mask nc)
        'out_nc_path': '',                         # output nc (i.e. t1a nc)
        'in_base_start_year': 1990,                # base start year
        'in_base_end_year': 2005,                  # base end year
        'in_comp_start_year': 2005,                # comparison start year
        'in_comp_end_year': 2020,                  # comparison end year
        'in_analysis_months': [9, 10, 11],         # analysis months
        'in_tmf': 2.0,                             # magnitude threshold
        'in_stand_qupper': 0.99,                   # upper ivt value
        'in_stand_qlower': 0.05,                   # lower ivt value
        'in_fmask_flags': 'Valid;Snow;Water',      # fmask flag values
        'in_max_cloud': 10,                        # max cloud percentage
        'in_interpolate': None,                    # interpolate missing pixels
        'in_add_result_to_map': True,              # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default interpolate (True)
    msg = 'Running Test Nine: default interpolate (True).'
    params = inputs.copy()
    params.update({'in_interpolate': True})
    ts.append(['t_9_def.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: interpolate is False
    msg = 'Running Test Nine: interpolate is False.'
    params = inputs.copy()
    params.update({'in_interpolate': False})
    ts.append(['t_9_a.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    

    # func: interpolate is None
    msg = 'Running Test Nine: interpolate is None (default to True).'
    params = inputs.copy()
    params.update({'in_interpolate': None})
    ts.append(['t_9_b.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])       
            
    return ts

### Test Nine: Run!

In [ ]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_nine_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_corrs=nc_corrs, tests=tests, verbose=False)

### Test Ten: Add To Map

In [98]:
def build_test_ten_funcs(in_nc, temp_nc):
    """sets up test ten functions"""
    
    # set default params for tool
    inputs = {
        'in_raw_nc_path': '',                      # input like nc (i.e. temp nc)
        'in_mask_nc_path': '',                     # input mask nc (i.e. temp mask nc)
        'out_nc_path': '',                         # output nc (i.e. t1a nc)
        'in_base_start_year': 1990,                # base start year
        'in_base_end_year': 2005,                  # base end year
        'in_comp_start_year': 2005,                # comparison start year
        'in_comp_end_year': 2020,                  # comparison end year
        'in_analysis_months': [9, 10, 11],         # analysis months
        'in_tmf': 2.0,                             # magnitude threshold
        'in_stand_qupper': 0.99,                   # upper ivt value
        'in_stand_qlower': 0.05,                   # lower ivt value
        'in_fmask_flags': 'Valid;Snow;Water',      # fmask flag values
        'in_max_cloud': 10,                        # max cloud percentage
        'in_interpolate': True,                    # interpolate missing pixels
        'in_add_result_to_map': None,              # add result to map
    }
    
    # set up list
    ts = []
            
    # func: default add result to map (True)
    msg = 'Running Test Ten: default add result to map (True).'
    params = inputs.copy()
    params.update({'in_add_result_to_map': True})
    ts.append(['t_10_def.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])
    
    # func: add result to map is False
    msg = 'Running Test Ten: add result to map is False.'
    params = inputs.copy()
    params.update({'in_add_result_to_map': False})
    ts.append(['t_10_a.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])    

    # func: add result to map is None
    msg = 'Running Test Ten: add result to map is None (default to True).'
    params = inputs.copy()
    params.update({'in_add_result_to_map': None})
    ts.append(['t_10_b.nc', arcpy.GDVSpectra_CVA_toolbox , params, msg])       
            
    return ts

### Test Ten: Run!

In [99]:
# build lsit of nc corruptors and tests to iterate
nc_corrs = build_nc_corruptors(temp_nc)
tests = build_test_ten_funcs(in_nc=nc_file, temp_nc=temp_nc)

# run!
run_corruptors_through_tests(in_nc=nc_file, nc_corrs=nc_corrs, tests=tests, verbose=False)

Corrupting NetCDF via: nc_default.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
No changes, setting up for default dataset.
Running Test Ten: default add result to map (True).
Performing test: nc_default_t_10_def.nc.
Running Test Ten: add result to map is False.
Performing test: nc_default_t_10_a.nc.
Running Test Ten: add result to map is None (default to True).
Performing test: nc_default_t_10_b.nc.
All tests applied to corruptor NetCDF.


